In [2]:
import re
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
# CSV 파일 로드
df = pd.read_csv('labels_updated.csv')

# 'filename' 열에서 파일 이름만 추출하여 업데이트
df['filename'] = df['filename'].apply(lambda x: os.path.basename(x))

In [8]:
print(df)

                                               filename     label
0     15284a77b3979147794bb67ed52c6c308b6fb41ed8c813...  positive
1     15284a77b3979147794bb67ed52c6c308b6fb41ed8c813...  positive
2     064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...  positive
3     064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...  positive
4     064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...  positive
...                                                 ...       ...
1666  bd210c67f8f6d1d3b34e1f724f26a0460a5e683af5c500...  negative
1667  bd210c67f8f6d1d3b34e1f724f26a0460a5e683af5c500...  negative
1668  6d88d862df110f113ce1fe29de6e61f87f40c942ca530a...  negative
1669  6d88d862df110f113ce1fe29de6e61f87f40c942ca530a...  negative
1670  f9b6f83a897ba44fd62aa9b81880f4472c1e88f457977e...  negative

[1671 rows x 2 columns]


In [9]:
# 데이터 디렉토리 생성 (이미 존재한다면 생성하지 않음)
os.makedirs('./data/train', exist_ok=True)
os.makedirs('./data/val', exist_ok=True)
os.makedirs('./data/test', exist_ok=True)

In [10]:
labels = ['positive', 'negative', 'neutral']
dfs_train_updated = []
dfs_val_updated = []
dfs_test_updated = []

def remove_korean_and_spaces(original_filename):
    # 한글과 공백을 제외한 문자만 남기고 모두 제거한 후 새 파일 이름 생성
    new_filename = re.sub(r'[가-힣\s]+', '', original_filename).strip()
    return new_filename

for label in labels:
    df_label = df[df['label'] == label]
    
    # train, validation, test 분할 (8:1:1)
    df_train, df_temp = train_test_split(df_label, test_size=0.2, random_state=42)
    df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

    def move_files(df_subset, subset_name):
        updated_rows = []
        for _, row in df_subset.iterrows():
            original_filename = row['filename']
            new_filename = remove_korean_and_spaces(original_filename)  # 한글과 공백 제거된 새 파일 이름
            
            new_path = f'./data/{subset_name}/{new_filename}'
            
            original_path = os.path.join('./sample_data/preprocessed_images', original_filename)  # 파일이 원래 저장된 경로
            
            # 파일이 실제로 존재하는지 확인
            if os.path.exists(original_path):
                shutil.copy(original_path, new_path)  # 파일 복사
                row['filename'] = new_filename  # 경로 대신 새 파일 이름 저장
            else:
                print(f"파일을 찾을 수 없습니다: {original_path}, 건너뜁니다.")
                continue
            
            updated_rows.append(row)
        return pd.DataFrame(updated_rows)
    
    dfs_train_updated.append(move_files(df_train, 'train'))
    dfs_val_updated.append(move_files(df_val, 'val'))
    dfs_test_updated.append(move_files(df_test, 'test'))

df_train_all = pd.concat(dfs_train_updated).reset_index(drop=True)
df_val_all = pd.concat(dfs_val_updated).reset_index(drop=True)
df_test_all = pd.concat(dfs_test_updated).reset_index(drop=True)

df_train_all.to_csv('./data/labels_train.csv', index=False)
df_val_all.to_csv('./data/labels_val.csv', index=False)
df_test_all.to_csv('./data/labels_test.csv', index=False)
